# 系统移植的意义


> 在基于 ARM 处理器的开发板上安装 Linux 系统

1. 是在 ARM 处理器上安装一个操作系统，需要特别注意系统文件的兼容性，不同架构的处理器指令集不兼容，即便是相同的处理器架构，板卡不同驱动代码也不兼容
2. 安装的是 Linux 系统，Linux 因其精简性特别适合于成本有限的嵌入式系统上部署。Linux 是一个通用的内核并不是为某一个特定的处理器架构或板卡设计的，所以从官方获取 Linux 源码后我们要先经过相应的配置使其与我们当前的硬件平台相匹配后才能进行编译和安装

> 操作系统有什么用？

操作系统：向下管理硬件、向上提供接口

操作系统为我们提供了： 1.进程管理 2.内存管理 3.网络接口 4.文件系统 5.设备管理

如果不用操作系统的话，在裸机开发的时候，我们什么函数都调不了，什么都要自己实现。这样做开发的时候需要大量的时间。只能做一些小的项目。


# Linux 系统移植的过程


1. 准备 Linux 内核镜像、SD 卡启动盘（uboot）
2. 通过拨码开关选择启动方式（SD 启动）
3. 通过 SD 卡中的引导程序安装系统
4. 安装 Linux 驱动程序
5. 安装 Linux 应用程序


# FS4412 开发板的启动过程


![alt text](image-1.png)

> 相关概念的解释

> > iROM :

"iROM"通常指的是内部只读存储器（Internal Read-Only Memory）。在嵌入式系统中，iROM 通常用于存储固定的程序代码或数据，这些内容在制造过程中被烧录到芯片中，运行时不能被修改。这种存储器的主要优点是它可以保护存储的内容不被意外或恶意修改。

>> BL0 

BL0: BL0 全名  boot loader ，也就是引导装载程序，是直接固化在内部 ROM 上的一小段程序，在上电的时候会默认去执行

执行的内容：

1. 禁用看门狗计时器
2. 初始化指令缓存 cache
3. 初始化栈区域
4. 初始化堆区域
5. 初始化块设备复制函数
6. 初始化锁相环并设置系统时钟
7. 将 BL1 复制到内部 SRAM 区域
8. 验证 BL1 的校验和，如果校验和失败，iROM 将尝试第二次引导（去 SD/MMC 通道 2）
9. 检查它是否处于安全启动模式，如果安全密钥值是用 S5PV210 编写的，那么它就是 secure-boot 模式。如果是安全启动模式，请验证 BL1 的完整性。
10. 跳转到 BL1 的起始地址

>> RAM

"RAM"是"Random Access Memory"的缩写，中文名为"随机存取存储器"。它是一种计算机存储器，可以在任何时间读取或写入数据。RAM 中的数据在断电后会丢失，因此它通常用于存储运行中的程序和临时数据。

> > EMMC

"eMMC"是"embedded MultiMediaCard"的缩写，中文名为"嵌入式多媒体卡"。它是一种闪存存储标准，通常用于智能手机、平板电脑和其他便携式设备中。

eMMC 的主要优点是体积小、成本低，适合于移动设备。但其读写速度和寿命相比于 SSD 等其他类型的闪存存储器要低。

相当于电脑外存，断电不丢失

>> rootfs 根文件系统

>> dtb 设备树文件

>> linux linux 系统镜像

>> uboot 启动引导程序

> 启动过程

1. 开发板上电后首先运行 SOC 内部 iROM 中固化的代码(BL0)，这段代码先对基本的软硬件环境(时钟等...)进行初始化，然后再检测拨码开关位置获取启动方式，然后再将对应存储器中的 uboot 搬移到内存，然后跳转到 uboot 运行。
2. uboot 开始运行后首先对开发板上的软硬件环境做进一步初始化（栈、网卡、硬件……），然后将 linux 内核、设备树(dtb)、根文件系统(rootfs)从外部存储器（或网络）搬移到内存，然后跳转到 linux 运行。
3. linux 开始运行后先对系统环境做初始化，当系统启动完成后，Linux 再从内存中（或网络）挂载根文件系统。（常用的挂载根文件系统的方式是用网络）


# Ubuntu 网络环境设置


</home/legendfantasy/C C++ code/系统移植/实验1 ubuntu网络环境配置.pdf>

# tftp服务器环境搭建

</home/legendfantasy/C C++ code/系统移植/实验2 tftp服务器环境搭建.pdf>

# NFS 服务器环境搭建

# uboot


## uboot 的基本情况


> Bootloader

在操作系统运行之前运行的一小段代码，用于将软硬件环境初始化到一个合适的状态，为操作系统的加载和运行做准备（其本身不是操作系统）

Bootloader 基本功能
-> 初始化软硬件环境
-> 引导加载 linux 内核
-> 给 linux 内核传参
-> 执行用户命令

bootloader 是启动引导程序的统称,嵌入式 linux 常用的 bootloader 是 uboot。

> uboot 特点

代码结构清晰
支持丰富的处理器与开发板，易于移植
支持丰富的用户命令
支持丰富的网络协议
支持丰富的文件系统
支持丰富的设备驱动
更新活跃、用户较多、资料丰富
开放源代码
较高的稳定性
不具有通用性（不同的处理器、开发板 uboot 不可通用）

> Uboot 源码结构

可以从官网获取 Uboot 开源代码包

![alt text](image-2.png)

uboot 并不是专为 fs4412 开发板而编写的工具，其中必然包含了很多为其他硬件平台适配的代码

> > 平台相关代码：

arch:与 CPU 架构相关的源代码

board:与开发板相关的源代码，包含各种官方评估板对应的源码,评估版就是这些公司在新出一款芯片时为这款芯片做的测试的板子

> > 平台无关代码

api： 这个目录下有很多的用户接口

common：这下面全是 uboot 命令的.c 文件

disk：磁盘操作的命令文件夹

drivers：这里面全是驱动文件

fs：文件系统，这里面每个文件夹内都是文件系统的源代码和 makefile 文件这里面 ext4 是嵌入式领域常用的文件系统。

include：这里是头文件

lib：这里是库

post：上电自检程序

net：网络文件夹，这里面是 uboot 支持的一些网络

> 配置文件、帮助文档、示例程序、工具等

boards.cfg:这个文件是配置信息

config.mk：也是一个配置文件

COPYING：这是版权文件虽然是开源软件但是也不能乱用

CREDITS：这里是代码贡献者名单包括他们的邮箱

readme：一个说明书或者说帮助文档，这个说明书有 5000 多行都是一些介绍性的的东西。

dos：这里是详细的说明书

dts：设备树文件夹，后面学习驱动时在详细学习。

examples：里面是例程，在不会写的时候可以参考一下

makefile：方便我们编译 uboot 镜像。它这个 makefile 是层层调用的，每个文件夹下都有，使用 make 命令后，总的 makefile 文件会调用每层的 makefile 文件来共同编译出 uboot 镜像。这个 makefile 文件非常的复杂。
tools：工具


## uboot 配置与编译


> 为什么要这么做？

不同的开发板的硬件平台不同，所使用的 uboot 代码也会不同，为了保证 uboot 适用所有的开发板，uboot 把所有的开发板都写出来了，需要哪个编译哪个

首先，编译源代码要在 ARM 机器上运行就需要使用 ARM 的编译器
其次，自己的硬件设备肯定不在官方提供的代码中，所以我们只能找一些相近的开发板作修改
这个过程也就是系统移植的过程

> uboot 配置

> > 指定编译 uboot 源码使用的编译器

为了我们编译出来的代码能为 ARM 处理器识别，需要用 arm 环境下的编译器

在 uboot 源码顶层目录下的 Makefile 中指定(CROSS_COMPILE 变量)

因为编译过程很复杂所有我们不可能自己去 gcc 编译。我们用官方写好的 makefile 就行。

但是我们要先指定编译器

打开顶层目录的 makefile 文件，搜索 CROSS_COMPILE,修改成如下：

![alt text](image-4.png)

> > 添加 Board 信息

这里我们参考的是 samsung 公司的 origen,首先把相关的代码拷贝并重命名为 fs4412，接着将 makefile 中的相关信息也作修改，使 make 命令能够顺利编译相关文件。

> uboot 编译

> > 指定当前使用的硬件平台

make <board_name>\_config

注 1：<board_name>为当前使用的开发板的名字

注 2：执行该命令的前提是 uboot 源码支持该开发板

注 3：该命令必须在 uboot 源码的顶层目录下执行

该命令是为注册我们修改的开发板的信息

make

注 1：该命令必须在 uboot 源码的顶层目录下执行

注 2：该命令执行后在 uboot 源码顶层目录下生成 u-boot.bin

该命令执行以编译 makefile 中写好的相关文件

编译完成后会在源码顶层目录下生成 u-boot.bin 文件，但该文件还不能在我们的开发板上运行，因为以上操作我们只是把 origen 相关的文件的名字改成了 fs4412，使 uboot 能 识别 fs4412 开发板，但文件中的代码还是 origen 的，和我们的开发板不匹配，所以我 们还需要进一步进行修改和配置

> 添加三星加密引导程序

考虑芯片启动的安全性，三星公司没有全部使用 uboot 的开源程序，有一部三星自己写好了，我们可以用但是不能看里面是什么 Exynos4412 需要三星提供的初始引导加密后我们的 u-boot 才能被引导运行，所以我们需要在 uboot 源码中添加三星提供的加密处理代码

> 添加编译脚本

使用 make 命令编译时只链接 uboot 源码中的相关代码，而我们添加的初始引导加密的 代码不会被连接到 u-boot.bin 中，所以这里我们自己编写编译脚本 build.sh，这个脚本 中除了对 uboot 源码进行配置和编译外还将初始引导加密代码链接到了 u-boot.bin 上， 最终生成一个完成的 uboot 镜像 u-boot-fs4412.bin

> UART 功能移植

虽然生成的 uboot 已经能在开发板上加载运行，但是此时的 uboot 还不能在终端上打印信息，原因在于 uboot 源码中对 UART 的配置与我们实际的硬件不匹配，需要修改 UART 源码

> 网卡功能移植

虽然可以通过终端输入命令，但此时的 uboot 还不能使用 ping、tftp 等命令，原因在于 命令都是操作网络的，而 uboot 源码中网卡的相关配置与我们当前的板子不匹配，所以 我们还要对网卡进行移植

> EMMC 移植

因为 uboot 源码中对 EMMC 的配置与我们的板子不匹配，这里还需要对 EMMC 相关的代码进行修改和配置

> 电源管理移植

因为 uboot 源码中对电源管理芯片的配置与我们的板子不匹配，后续有可能会导致内核启动卡死，这里还需要对电源管理芯片相关的代码进行修改和配置

以上 uboot 的配置和编译过程详见实验手册


## 使用 SD 卡刷入 uboot 程序以制作启动盘



## uboot 的使用


> uboot 模式

自启动模式
uboot 启动后若没有用户介入，倒计时结束后会自动执行自启动
环境变量(bootcmd)中设置的命令（一般作加载和启动内核）
如果没有安装操作系统，则会一直显示超时
交互模式
倒计时结束之前按下任意按键 uboot 会进入交互模式，交互模式下
用户可输入 uboot 命令

> uboot 环境变量命令

1. help  
   查看 uboot 支持的所有命令
2. help 命令  
   查看当前命令的使用方法

3. printenv  
   打印 uboot 中所有的环境变量
4. setenv  
   设置指定的环境变量（保存在 RAM 中）
   `setenv 环境变量 环境变量的值`
   如果想清除一个环境变量的值，那么就将环境变量值的部分就置空

5. saveenv  
   保存所有环境变量到 EMMC 中

6. ipaddr  
   uboot 的 IP 地址

7. serverip  
   服务器的 IP 地址（即 ubuntu 的 IP）

8. bootdelay  
   进入自启动模式之前倒计时的秒数

9. loadb  
   通过 Kermit 协议下载文件到指定的内存地址  
    `loadb  地址`
10. tftp  
    通过 tftp 协议下载文件到指定的内存地址  
     `tftp  地址  文件名`

注：使用 tftp 之前要配置好网络及 tftp 服务器

11. mmc read  
    将 EMMC 中指定扇区中的内容读取到内存中指定的地址  
    `mmc read  <addr> <blk#> <cnt>`  
     addr: 内存地址  
     blk#: EMMC 中的扇区编号  
     cnt: 读取的扇区的个数

12. mmc write  
    将内存中指定地址中的内容写入到 EMMC 中指定的扇区  
    `mmc write <addr> <blk#> <cnt>`

13. bootcmd  
    自启动的环境变量  
    该环境变量可以设置成一到多个 uboot 命令的集合（若有多个使用 `\;`分割）  
    自启动模式下 uboot 就会按照 bootcmd 中命令的顺序逐条执行


通过前面的一系列操作得到我们自己编译好的 uboot 系统启动程序后，我们接下来需要制作像 windows 下安装操作系统那样的 u 盘启动盘

![alt text](image-5.png)

SD 卡的存储以扇区为单位,每个扇区的大小为 512Byte, 其中零扇区存储分区表（即分区信息）,后续的扇区可自行分区和格式化；

若选择 SD 卡启动，处理器上电后从第一个扇区开始将其中的内容搬移到内存，所以我们把 uboot 放到从第一个扇区开始之后的空间， 之后的空间根据个人需求可进行分区和格式化

> 制作一个空镜像

需要为 0 分区预留出空间，所以我们需要制作一个空镜像刷入。

在 Ubuntu 下执行命令：
`sudo dd if=/dev/zero of=zero.bin count=1`

> 制作一个空镜像来清除原本 SD 卡上的内容

在 Ubuntu 下执行命令：
`sudo dd if=/dev/zero of=clear.bin count=2048`

> 使用工具把做好的镜像刷入


# linux 内核


## Linux 内核的基本情况


> 内核

内核是一个操作系统的核心，提供了操作系统最基本的功能，是操作系统工作的基础，决定着整个系统的性能和稳定性

> 操作系统

操作系统是在内核的基础上添加了各种工具集、桌面管理器、库、shell、应用程序等

![alt text](image-6.png)

> linux 层次结构

![alt text](image-7.png)

> linux 内核特点

代码结构清晰、模块化设计
支持丰富的硬件平台
较高的稳定性
轻量化及较强的裁剪性
开放源代码
更新活跃、用户较多、资料丰富
支持丰富的网络协议

> linux 内核源码结构

![alt text](image-8.png)

平台相关代码

arch: 与 CPU 架构相关的源代码

平台无关代码

block:磁盘设备的支持
crypto:加密相关  
drivers:设备驱动
firmware:固件  
fs:文件系统
include:头文件  
init:内核初始化
ipc:进程间通信  
kernel:内核核心调度机制等
lib:库  
mm:内存管理
net:网络协议  
scripts:工具、脚本等
security:安全  
usr:打包与压缩
virt:虚拟

帮助文档、示例程序、工具等

COPYING: 版权  
CREDITS: 内核贡献者  
README: 说明文档  
Documentation: 帮助文档  
Makefile: 编译管理  
samples: 示例  
tools: 工具


## linux 内核源码的配置和编译


> linux 内核源码配置

> > 指定处理器架构及编译工具

在 Linux 内核源码顶层目录下的 Makefile 中指定(ARCH、CROSS_COMPILE)

> > 导入当前处理器的默认配置

make <soc_name>\_defconfig

即使是都是 ARM 处理器，不一样的架构汇编也不同。所以光指定叫 arm 肯定不行，因为再 arc/arm 下只有叫 exynos 的没有 exynos4412

注 1：soc_name 为当前使用的处理器的名字

注 2：内核源码的 arch/arm/configs 下对各个厂商的 soc 都有一个默认配置文件

执行 $ make exynos_defconfig 命令，执行完成后终端会显示：configuration written to .config
这个.config 保存了对 linux 内核功能的设置

执行该命令后就会将对应的配置文件中的信息导入到源码顶层目录下的.config

文件中 CONFIG_xxx=y 表示内核选中了该功能，内核编译时就会将该功能对应的代码编译，内核的体积也会增大。#CONFIG_xxx is not set 表示内核没有选中该功能，内核编译时该功能对应的代码不会被编译，内核的体积也会减小。

> > 修改配置

默认配置只能保证内核拥有最基本的功能，我们需要根据自己的实际需求对内核做进一步的配置

方法 1：

直接修改.config 文件（不推荐）

太多了不好找，并且很多功能是有依赖关系的，要很熟，把最上层到最底层所需的全部东西都打开才能实现一共功能。

方法 2：

make menuconfig

图形化配置界面，在该界面下我们可以对 linux 进行进一步的修改和配置方向键可选择不同的选项，‘Enter’键进入子菜单，‘Y’键选中某项功能， ‘N’键去除某项功能，‘M’键将该功能编译成内核模块，两次‘Esc’键退出界面，‘?’键为帮助选项，‘/’键为搜索选项

![alt text](image-9.png)

修改配置

[ ] 有两种状态

输入 Y，显示“\*”，内核中该功能被选中，相关代码会被编译进内核

输入 N，显示“ ”，内核中该功能不被选中，相关代码不会被编译进内核

< > 有三种状态

输入 Y，显示“\*”，内核中该功能被选中，相关代码会被编译进内核

输入 N，显示“ ”，内核中该功能不被选中，相关代码不会被编译进内核

输入 M，显示“M”，内核中该功能被选为模块（被编译为独立的模块）

注：使用 make mnuconfig 配置的本质还是修改.config 文件

> linux 内核编译

执行 make uImage 命令，会对 linux 内核进行编译，会在源码的 arch/arm/boot/目录下生成 uImage 镜像


## linux 内核安装与加载

1. 在tftp本地目录下放入
   1. exynos4412-fs4412.dtb 设备树文件
   2. uImage linux 内核镜像
   3. ramdisk.img 根文件系统镜像
2. 在uboot交互模式下设置uboot启动参数
   1. 设置开发板ip和服务器ip
   2. 设置bootcmd 
    setenv bootcmd   
    tftp 0x41000000 uImage;  
    tftp 0x42000000 exynos4412-fs4412.dtb;  
    tftp 0x43000000 ramdisk.img;  
    bootm 0x41000000 0x4300000 0x42000000  

# 设备树


## 设备树的基本情况


> 什么是设备树文件

设备树是一种描述硬件信息的数据结构，Linux 内核运行时可以通过设备树将硬件信息直接传递给 Linux 内核，而不再需要在 Linux 内核中包含大量的冗余编码

> 设备树文件

dts 设备树源文件

dtsi 类似于头文件，包含一些公共的信息，可被其它设备树文件引用

dtb 编译后的设备树文件

> 设备树语法

设备树的语法为树状结构，由一系列的节点和属性组成，根节点下包含子节点

子节点下还可以包含子节点，节点内部包含了对应设备的属性

```C
    /{
    	memory{
    		0x40000000 0x40000000
    	};
    	dm9000{
    		0x05000000;
    	};
    	key{
    		up{

    		};
    		down{

    		};
    	};
    }；
```

/就表示根节点每个子节点中有一些属性比如内存 memory 内又位置和大小。网卡 dm9000 里有网卡的位置。键盘 key 内有具体哪个按键。


## 设备树编译


内核源码中并没有 fs4412 平台的设备树文件，这里我们从源码支持的平台中找一个硬
件与我们最类似的，在其基础上进行修改，这里我们参考的是 samsung 公司的 origen

1. 首先拷贝 arch/arm/boot/dts 下的 exynos4412-origen.dts
2. 因为添加的设备树文件也要编译，所以对应的 Makefile 也要修改
3. 回到源码的顶层目录下编译设备树 $ make dtbs ，最终在 arch/arm/boot/dts/目录下生成了 exynos4412-fs4412.dtb


# linux 内核驱动移植


1. 在内核源码的顶层目录下执行如下命令，修改内核配置 $ make menuconfig，给内核选配 DM9000 网卡驱动，然后选择“Save”保存
2. 在设备树中添加网卡的硬件信息
3. 修改时钟相关配置（忽略无用的时钟）
4. 修改 EMMC 相关配置
5. 编译内核和设备树


# 根文件系统


## 根文件系统的基本情况


> 什么是根文件系统

根文件系统是内核启动后挂载的第一个文件系统系统引导程序会在根文件系统挂载后从中把一些基本的初始化脚本和服务等加载到内存中去运行

> 根文件系统的内容

![alt text](image-10.png)

1. bin 
    shell 命令(elf 格式)(通过 busybox 编译生成)  
    都是二进制文件，所以需要区分架构,他们都来自busybox。busybox可以理解成这些命令的源代码

2. dev 设备文件(内核启动后会将设备信息写入该目录)

3. etc 内核配置文件

4. lib 共享库(elf 格式)(从交叉编译工具链中获取)  
    为什么要有库呢，因为程序的执行需要动态库。不然链接动态库的应用程序不能再开发板上跑。而我们使用交叉编译的方式编辑和编译程序再 ubuntu 上。所以开发板不需要有静态库。这些库是怎么来的呢，首先 ubuntu 的库我们肯定用不了。他们都是 x86 架构的。不过交叉编译工具链里有库。我们把这里.so 结尾的也就是动态库复制过来

5. linuxrc 内核运行的第一个应用程序(通过 busybox 编译生成)  

   1. /linuxrc是一个可执行的应用程序

        1. /linuxrc是应用层的，和内核源码一点关系都没有

        2. /linuxrc在开发板当前内核系统下是可执行的。因此在ARM SoC的linux系统下，这个应用程序就是用arm-linux-gcc编译链接的；如果是在PC机linux系统下，那么这个程序就是用gcc编译连接的。

        3. /linuxrc如果是静态编译连接的那么直接可以运行；如果是动态编译连接的那么我们还必须给他提供必要的库文件才能运行。但是因为我们/linuxrc这个程序是由内核直接调用执行的，因此用户没有机会去导出库文件的路径，因此实际上这个/linuxrc没法动态连接，一般都是静态连接的。

    2. /linuxrc执行时引出用户界面

        1. 操作系统启动后在一系列的自己运行配置之后，最终会给用户一个操作界面（也许是cmdline，也许是GUI），这个用户操作界面就是由/linuxrc带出来的。

        2. 用户界面等很多事并不是在/linuxrc程序中负责的，用户界面有自己专门的应用程序，但是用户界面的应用程序是直接或者间接的被/linuxrc调用执行的。用户界面程序和其他的应用程序就是进程2、3、4·····，这就是我们说的进程1（init进程，也就是/linuxrc）是其他所有应用程序进程的祖宗进程。

    3. /linuxrc负责系统启动后的配置

       1. 就好像一个房子建好之后不能直接住，还要装修一样；操作系统启动起来后也不能直接用，要配置下。

       2. 操作系统启动后的应用层的配置（一般叫运行时配置，英文简写etc）是为了让我们的操作系统用起来更方便，更适合我个人的爱好或者实用性。

    4. /linuxrc在嵌入式linux中一般就是busybox

       1. busybox是一个C语言写出来的项目，里面包含了很多.c文件和.h文件。这个项目可以被配置编译成各个平台下面可以运行的应用程序。我们如果用arm-linux-gcc来编译busybox就会得到一个可以在我们开发板linux内核上运行的应用程序。

       2. busybox这个程序开发出来就是为了在嵌入式环境下构建rootfs使用的，也就是说他就是专门开发的init进程应用程序。

       3. busybox为当前系统提供了一整套的shell命令程序集。譬如vi、cd、mkdir、ls等。在桌面版的linux发行版（譬如ubuntu、redhat、centOS等）中vi、cd、ls等都是一个一个的单独的应用程序。但是在嵌入式linux中，为了省事我们把vi、cd等所有常用的shell命令集合到一起构成了一个shell命令包，起名叫busybox。

mnt 挂载目录(非必要)  
proc 进程相关文件(内核启动后会将进程信息写入该目录)  
root 超级用户家目录(非必要)  
sbin 系统管理 shell 命令(elf 格式)(通过 busybox 编译生成)  
sys 驱动相关文件(内核启动后会将驱动信息写入该目录)  
usr shell 命令(elf 格式)(通过 busybox 编译生成)


## 根文件系统编译

> 构建自己的根文件系统

1. 从官网获取busybox源代码
2. 在busybox目录下执行 make menuconfig 命令对busybox进行配置
3. 编译 busybox $ make
4. 安装 busybox
5. 将交叉编译工具链中的库文件拷贝到_install 目录下
6. 删除库文件中的静态库
7. 删除共享库中的符号表（需要在 root 用户下操作）
8. 将资料中“移植相关文件”下的 etc 目录（配置文件）拷贝到当前目录下
9. 创建其他目录$ mkdir dev mnt proc root systm pvar